In [1]:
!pip install transformers
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

# Load pre-trained model and tokenizer (GPT2 from Hugging_Face)
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Prepare dataset for training and testing
train_path = '/content/drive/My Drive/Colab_Notebooks/datasets/train_data.jsonl'
test_path = '/content/drive/My Drive/Colab_Notebooks/datasets/test_data.jsonl'

In [4]:
!pip install torch transformers[torch] accelerate -U


In [5]:
# Create a dataset from the training and test files
# TextDataset is a utility from transformers that reads and formats the data file for training
train_dataset = TextDataset(
  tokenizer=tokenizer, # The tokenizer used to process the text
  file_path=train_path, # The path to the training data
  block_size=128 # The size of the blocks the dataset will be split into for training
)

test_dataset = TextDataset(
  tokenizer=tokenizer, # The tokenizer used to process the text
  file_path=test_path, # The path to the testing data
  block_size=128 # The size of the blocks the dataset will be split into for testing
)

# Set up the data collator
# Data collator is responsible for batching the data and preparing it for input into the model
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, # The tokenizer used for preparing the data
    mlm=False, # Indicates whether masked language modeling is used. False for causal (autoregressive) language modeling with GPT-2
)

# Define the training arguments
# These specify various settings and hyperparameters for the training process
training_args = TrainingArguments(
    output_dir='./results', # Directory where the training outputs (like model checkpoints) will be saved
    overwrite_output_dir=True, # If True, overwrite the contents of the output directory if it already exists
    num_train_epochs=3, # The total number of training epochs (complete passes through the dataset)
    per_device_train_batch_size=4, # Batch size per device during training
    per_device_eval_batch_size=4, # Batch size for evaluation
    eval_steps=400, # Perform an evaluation every `eval_steps` steps
    save_steps=800, # Save a model checkpoint every `save_steps` steps
    warmup_steps=500, # Number of warmup steps for learning rate scheduler
    prediction_loss_only=True, # When True, only return the loss; otherwise, also return logits and more during evaluation
)

# Initialize the Trainer
trainer = Trainer(
    model=model, # The model to be trained
    args=training_args, # The training arguments
    data_collator=data_collator, # The data collator
    train_dataset=train_dataset, # The training dataset
    eval_dataset=test_dataset, # The evaluation (testing) dataset
)

# Start the training process
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=6, training_loss=2.9371121724446616, metrics={'train_runtime': 76.4344, 'train_samples_per_second': 0.235, 'train_steps_per_second': 0.078, 'total_flos': 1175814144000.0, 'train_loss': 2.9371121724446616, 'epoch': 3.0})